In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft

# ------------------------------
# Load Data
# ------------------------------
df = pd.read_csv("C:/Users/jay.jain/Desktop/Codes - Jay/Assignment 1/FEIcm1.csv")
df.columns = ['Ticker1', 'Ticker2', 'Description', 'Timestamp', 'Col5', 'Frequency', 
              'Open', 'High', 'Low', 'Close', 'Volume']

# Convert Timestamp to datetime and drop invalid rows
df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors='coerce')
df = df.dropna(subset=["Timestamp"])
df = df.sort_values("Timestamp")

# Filter data for years 2012 to 2015
df = df[(df["Timestamp"].dt.year >= 2012) & (df["Timestamp"].dt.year <= 2015)]

# Set timestamp as index
df.set_index("Timestamp", inplace=True)

# ------------------------------
# Resample to Daily Mean
# ------------------------------
daily_open = df["Open"].resample("D").mean().dropna()

# ------------------------------
# Perform Spectral Analysis (FFT)
# ------------------------------
signal = daily_open.values
n = len(signal)
fft_values = fft(signal)
power = np.abs(fft_values[:n//2])**2  # Only positive frequencies
freq = np.fft.fftfreq(n, d=1)[:n//2]  # d=1 means daily sampling

# Convert frequency to periods (days per cycle)
with np.errstate(divide='ignore', invalid='ignore'):
    periods = 1 / freq
    periods[~np.isfinite(periods)] = 0  # Replace inf or NaN with 0 for plotting

# ------------------------------
# Plot Power Spectrum
# ------------------------------
plt.figure(figsize=(12, 5))
plt.plot(periods, power, color='purple')
plt.xlim(0, 365)  # Focus on cycles up to yearly
plt.title("Spectral Analysis of Daily Open Price (2012–2015)", fontsize=14)
plt.xlabel("Period (Days per Cycle)")
plt.ylabel("Power (Amplitude²)")
plt.grid(True)
plt.tight_layout()
plt.show()